In [2]:
import numpy as np
import mnist
from keras.datasets import fashion_mnist
from pandas import *

Using TensorFlow backend.


In [22]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [4]:
B1 = 0.9
B2 = 0.999
EPS = 1e-8

class Adam:
    def __init__(self, nu = 1e-3):
        self.nu = nu
        self.m, self.v, self.t = 0, 0, 0
    
    def initialState(self, theta0):
        self.theta = theta0
        
    def optimize(self, g):
        self.m = B1 * self.m + (1 - B1) * g
        self.v = B2 * self.v + (1 - B2) * (g ** 2)
        self.t += 1
        mMean = self.m / (1 - B1 ** self.t)
        vMean = self.v / (1 - B2 ** self.t)
        self.theta = self.theta - self.nu * (mMean / (np.sqrt(vMean) - EPS))
        return self.theta

In [ ]:
class AdaGrad:
    def __init__(self, nu = 1e-3):
        self.G_sum = 0
    
    def initialState(self, theta0):
        self.eps = np.full(theta0.shape, EPS)
        
    def optimize(self, g):
        self.G_sum += g * g

        self.g -=  / np.sqrt(self.G_sum + EPS)
        return self.g

In [5]:
class Conv3x3:
    def __init__(self, filterCount, optimizer):
        self.filterCount = filterCount
        self.filters = np.random.randn(filterCount, 3, 3) / 9.0
        self.optimizer = optimizer
        optimizer.initialState(self.filters)
    
    def applyTo(self, image):
        h, w = image.shape
        output = np.zeros((h - 2, w - 2, self.filterCount))

        for i in range(h - 2):
            for j in range(w - 2):
                block = image[i : (i + 3), j : (j + 3)]
                output[i, j] = np.sum(block * self.filters, axis=(1, 2))

        self.lastImage = image
        return output

    def optimize(self, dL_dout):
        h, w = self.lastImage.shape
        
        dL_dfilters = np.zeros(self.filters.shape)
        for i in range(h - 2):
            for j in range(w - 2):
                block = self.lastImage[i : (i + 3), j : (j + 3)]
                for f in range(self.filterCount):
                    dL_dfilters[f] += dL_dout[i, j, f] * block

        self.filters = self.optimizer.optimize(dL_dfilters)

In [6]:
class MaxPool2:

    def applyTo(self, image):
        h, w, f = image.shape
        output = np.zeros((h // 2, w // 2, f))

        for i in range(h // 2):
            for j in range(w // 2):
                block = image[(2 * i) : (2 * i + 2), (2 * j) : (2 * j + 2)]
                output[i, j] = np.amax(block, axis=(0, 1))
        
        self.lastImage = image
        return output

    
    def optimize(self, dL_dout):
        h, w, f = self.lastImage.shape
        dL_din = np.zeros((h, w, f))

        for i in range(h // 2):
            for j in range(w // 2):
                block = self.lastImage[(2 * i) : (2 * i + 2), (2 * j) : (2 * j + 2)]
               
                h, w, f = block.shape
                mx = np.amax(block, axis=(0, 1))

                for i2 in range(h):
                    for j2 in range(w):
                        for f2 in range(f):
                            if block[i2, j2, f2] == mx[f2]:
                                dL_din[2 * i + i2, 2 * j + j2, f2] = dL_dout[i, j, f2]
        return dL_din

In [17]:
class Softmax:
    def __init__(self, length, nodes, initOptimizer):
        self.weights = np.random.randn(length, nodes) / length
        self.biases = np.zeros(nodes)

        self.weightsOptimizer = initOptimizer()
        self.weightsOptimizer.initialState(self.weights)

        self.biasesOptimizer = initOptimizer()
        self.biasesOptimizer.initialState(self.biases)
        
    def applyTo(self, image):
        self.lastImageShape = image.shape
        image = image.flatten()
        length, nodes = self.weights.shape
        totals = np.dot(image, self.weights) + self.biases
        exp = np.exp(totals)
        
        self.lastImage = image
        self.lastTotals = totals

        return exp / np.sum(exp, axis=0)

    def optimize(self, dL_dout):
        for i, grad in enumerate(dL_dout):
            if grad == 0:
                continue

            totalExp = np.exp(self.lastTotals)
            s = np.sum(totalExp)

            dout_dtotals = -totalExp[i] * totalExp / (s ** 2)
            dout_dtotals[i] = totalExp[i] * (s - totalExp[i]) / (s ** 2)

            dtotals_dweights = self.lastImage
            dtotals_dbiases = 1
            dtotals_dimages = self.weights

            dL_dtotals = grad * dout_dtotals

            dL_dweights = np.matmul(dtotals_dweights[np.newaxis].T, dL_dtotals[np.newaxis])
            dL_dbiases = dL_dtotals * dtotals_dbiases
            dL_dimages = np.matmul(dtotals_dimages, dL_dtotals)

                        
            self.weights = self.weightsOptimizer.optimize(dL_dweights)
            self.biases = self.biasesOptimizer.optimize(dL_dbiases)

            return dL_dimages.reshape(self.lastImageShape)

In [18]:
H, W = train_images[0].shape
D = 4
LABELS = 10

In [19]:
class Model:
    def __init__(self, *layers):
        self.layers = layers
        
        
    def _processImage(self, image):
        image = (image / 255) - 0.5
        for layer in self.layers:
            image = layer.applyTo(image)
        return image
        
    def _optimize(self, output, c):
        grad = np.zeros(10)
        grad[c] = -1 / output[c]

        for layer in reversed(self.layers):
            grad = layer.optimize(grad)
        
    def fit(self, images, classes):
        totalLoss, totalCorrect = 0, 0
        
        for i, (image, c) in enumerate(zip(images, classes)):
            if i % 100 == 0:
                print('Step: {0}, Loss: {1}, Accuracy: {2}%'.format(i, totalLoss / 100, totalCorrect))
                totalLoss, totalCorrect = 0, 0
                
            output = self._processImage(image)
            self._optimize(output, c)
            
            totalLoss -= np.log(output[c])
            totalCorrect += (np.argmax(output) == c)
    
    def classify(self, image):
        output = self._processImage(image)
        return np.argmax(output)

In [ ]:
conv = Conv3x3(D, Adam())                      # H x W x 1 --> H-2 x W-2 x D
pool = MaxPool2()                              # H-2 x W-2 x D --> (H-2)/2 x (W-2)/2 x D
softmax = Softmax(((H-2) // 2) * ((W-2) // 2) * D, LABELS, Adam) # (H-2)/W x (W-2)/2 x D --> (H-2)/2*(W-2)/w*D

m = Model(conv, pool, softmax)
permutation = np.random.permutation(len(train_images))
m.fit(train_images[permutation], train_labels[permutation])

Step: 0, Loss: 0, Accuracy: 0%
Step: 100, Loss: 2.44349976717, Accuracy: 8%
Step: 200, Loss: 2.30667343897, Accuracy: 17%
Step: 300, Loss: 2.30487804472, Accuracy: 14%
Step: 400, Loss: 2.29236341358, Accuracy: 13%
Step: 500, Loss: 2.27364836326, Accuracy: 22%
Step: 600, Loss: 2.26768354295, Accuracy: 16%
Step: 700, Loss: 2.2728038169, Accuracy: 16%
Step: 800, Loss: 2.23649550482, Accuracy: 21%
Step: 900, Loss: 2.24007475566, Accuracy: 20%
Step: 1000, Loss: 2.22294346872, Accuracy: 21%
Step: 1100, Loss: 2.22828542948, Accuracy: 22%
Step: 1200, Loss: 2.1900990903, Accuracy: 26%
Step: 1300, Loss: 2.22701982157, Accuracy: 22%
Step: 1400, Loss: 2.22898101871, Accuracy: 22%
Step: 1500, Loss: 2.13920389747, Accuracy: 34%
Step: 1600, Loss: 2.19037398608, Accuracy: 20%
Step: 1700, Loss: 2.12003698964, Accuracy: 26%
Step: 1800, Loss: 2.16548205066, Accuracy: 23%
Step: 1900, Loss: 2.14350158851, Accuracy: 24%
Step: 2000, Loss: 2.16767442975, Accuracy: 21%
Step: 2100, Loss: 2.09130302404, Accuracy

Step: 17400, Loss: 1.71072843861, Accuracy: 40%
Step: 17500, Loss: 1.89547249071, Accuracy: 31%
Step: 17600, Loss: 2.00675975901, Accuracy: 32%
Step: 17700, Loss: 1.84190730132, Accuracy: 35%
Step: 17800, Loss: 1.95845464209, Accuracy: 31%
Step: 17900, Loss: 1.85317100716, Accuracy: 38%
Step: 18000, Loss: 1.80350326605, Accuracy: 38%
Step: 18100, Loss: 1.80346183479, Accuracy: 38%
Step: 18200, Loss: 1.89464649721, Accuracy: 40%
Step: 18300, Loss: 1.80628849595, Accuracy: 36%
Step: 18400, Loss: 1.71983029448, Accuracy: 45%
Step: 18500, Loss: 1.96506356244, Accuracy: 29%
Step: 18600, Loss: 1.82839244021, Accuracy: 28%
Step: 18700, Loss: 1.59271477648, Accuracy: 42%
Step: 18800, Loss: 1.7763437486, Accuracy: 35%
Step: 18900, Loss: 1.88392436256, Accuracy: 38%
Step: 19000, Loss: 1.8256401032, Accuracy: 33%
Step: 19100, Loss: 1.77275531424, Accuracy: 39%
Step: 19200, Loss: 1.74276245583, Accuracy: 35%
Step: 19300, Loss: 1.93540698677, Accuracy: 29%
Step: 19400, Loss: 1.8328771819, Accuracy:

Step: 34600, Loss: 1.82906802164, Accuracy: 41%
Step: 34700, Loss: 1.92321003635, Accuracy: 30%
Step: 34800, Loss: 1.86874743769, Accuracy: 40%
Step: 34900, Loss: 1.67339996706, Accuracy: 37%
Step: 35000, Loss: 1.78848416369, Accuracy: 35%
Step: 35100, Loss: 1.83537415466, Accuracy: 35%
Step: 35200, Loss: 1.82701385936, Accuracy: 39%
Step: 35300, Loss: 1.92034779759, Accuracy: 33%
Step: 35400, Loss: 1.83134292767, Accuracy: 35%
Step: 35500, Loss: 1.89508169439, Accuracy: 31%
Step: 35600, Loss: 1.75950385225, Accuracy: 42%
Step: 35700, Loss: 1.80153935213, Accuracy: 34%
Step: 35800, Loss: 1.59196688204, Accuracy: 44%
Step: 35900, Loss: 1.93847458801, Accuracy: 34%
Step: 36000, Loss: 1.96001154576, Accuracy: 30%
Step: 36100, Loss: 1.54742937696, Accuracy: 43%
Step: 36200, Loss: 1.72628179946, Accuracy: 39%
Step: 36300, Loss: 1.75199463555, Accuracy: 35%
Step: 36400, Loss: 1.67571044327, Accuracy: 42%
Step: 36500, Loss: 1.67722914318, Accuracy: 41%
Step: 36600, Loss: 1.89221036656, Accura

In [11]:
prediction = [m.classify(image) for image in test_images]


testCount = len(test_images)
correct = sum([prediction[i] == test_labels[i] for i in range(len(prediction))])
print('Accuracy: {0}%'.format(correct * 100 / testCount))

Accuracy: 20%


In [12]:
cm = [[0 for j in range(10)] for i in range(10)]
for p, l in zip(prediction, test_labels):
    cm[l][p] += 1

print(DataFrame(cm))

   0   1  2  3   4   5   6   7  8   9
0  0   4  0  0   2   5  70   4  0   0
1  0  50  0  0   0   4  66   1  2   3
2  0  13  0  0   1   3  95   2  2   0
3  0  10  0  0   3   8  80   3  0   3
4  0  13  0  0  16   3  70   8  0   0
5  0  18  0  0   1  12  51   3  1   1
6  0   5  0  0   1   1  80   0  0   0
7  0   4  0  0   2   1  67  19  2   4
8  0  16  0  0   3   8  51   2  5   4
9  0   6  0  0   1   0  61   5  0  21


In [13]:
(x_train_fmnist, y_train_fmnist), (x_test_fmnist, y_test_fmnist) = fashion_mnist.load_data()

x_train_fmnist = x_train_fmnist[:1000]
y_train_fmnist = y_train_fmnist[:1000]
x_test_fmnist = x_train_fmnist[:1000]
y_test_fmnist = y_train_fmnist[:1000]

In [14]:
m = Model(conv, pool, softmax)
permutation = np.random.permutation(len(x_train_fmnist))
m.fit(x_train_fmnist[permutation], y_train_fmnist[permutation])

Step: 0, Loss: 0, Accuracy: 0%
Step: 100, Loss: 2.31526482629, Accuracy: 11%
Step: 200, Loss: 2.28081805383, Accuracy: 11%
Step: 300, Loss: 2.21332384969, Accuracy: 23%
Step: 400, Loss: 2.20124129264, Accuracy: 23%
Step: 500, Loss: 2.11493267216, Accuracy: 26%
Step: 600, Loss: 2.15959366146, Accuracy: 25%
Step: 700, Loss: 2.20439999548, Accuracy: 21%
Step: 800, Loss: 2.16469881401, Accuracy: 26%
Step: 900, Loss: 2.07221354315, Accuracy: 26%


In [15]:
prediction = [m.classify(image) for image in x_test_fmnist]


testCount = len(test_images)
correct = sum([prediction[i] == y_test_fmnist[i] for i in range(len(prediction))])
print('Accuracy: {0}%'.format(correct * 100 / testCount))

Accuracy: 33%


In [16]:
cm = [[0 for j in range(10)] for i in range(10)]
for p, l in zip(prediction, y_test_fmnist):
    cm[l][p] += 1

print(DataFrame(cm))

   0   1   2   3   4  5   6   7   8   9
0  8  16   1   5   1  0  22  10  43   1
1  0  55   1   0   0  0   3  14  31   0
2  0   6  19   1   5  0   8  16  31   0
3  0   9   0  19   1  0   4  17  42   0
4  0   7   4   0  18  0  16   9  41   0
5  0   3   0   0   0  0   0  51  43   3
6  1  13   2   3   5  0  21  12  42   1
7  0   4   0   0   0  0   0  88  23   0
8  0   2   0   0   1  0   1  12  86   0
9  0   3   0   0   0  0   0  24  51  21
